<a href="https://colab.research.google.com/github/yunhyeongil/pandas_base/blob/main/hw_ML2DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

# 데이타 수집 및 전처리

In [3]:
df = pd.read_csv('/content/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
print('열의 갯수 : ', len(df.columns))
print(df.columns)

열의 갯수 :  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [5]:
df['headline'].isnull().values.any()

False

In [7]:
# headline열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장하자
headline = []
headline.extend(list(df.headline.values)) #헤드라인의 값들을 리스트로 저장
print(df.headline.values)

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell'
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.'
 'The New Noma, Explained' ...
 'Gen. Michael Hayden Has One Regret: Russia'
 'There Is Nothin’ Like a Tune' 'Unknown']


In [9]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [10]:
print('총 샘플의 갯수 : {}'.format(len(headline)))

총 샘플의 갯수 : 1324


# 노이즈 제거

In [11]:
headline = [n for n in headline if n != "Unknown"]
print('노이즈 값 제거 후 샘플의 갯수 : {}'.format(len(headline)))

노이즈 값 제거 후 샘플의 갯수 : 1214


In [12]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [13]:
def repreprocessing(s):
  s = s.encode("utf8").decode("ascii", 'ignore')
  return ''.join(c for c in s if c not in punctuation).lower() # 구두점제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

# 토큰화

In [17]:
t= Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)
print(t.word_index)

단어 집합의 크기 : 3494
{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'is': 8, 'on': 9, 'with': 10, 'trump': 11, 'as': 12, 'at': 13, 'new': 14, 'how': 15, 'from': 16, 'it': 17, 'an': 18, 'that': 19, 'be': 20, 'season': 21, 'us': 22, 'you': 23, 'its': 24, 'what': 25, 'episode': 26, 'can': 27, 'your': 28, 'not': 29, 'he': 30, 'now': 31, 'his': 32, 'are': 33, 'teaching': 34, 'war': 35, 'out': 36, 'no': 37, 'was': 38, 'by': 39, 'trumps': 40, 'has': 41, 'over': 42, 'may': 43, 'into': 44, 'why': 45, 'more': 46, 'we': 47, 'who': 48, 'about': 49, 'recap': 50, 'activities': 51, '1': 52, 'just': 53, 'do': 54, 'women': 55, 'when': 56, 'syria': 57, 'trade': 58, 'i': 59, '2': 60, 'or': 61, 'will': 62, 'this': 63, 'have': 64, 'president': 65, 'but': 66, 'home': 67, 'up': 68, 'long': 69, 'one': 70, 'off': 71, 'facebook': 72, 'house': 73, 'gop': 74, 'our': 75, 'case': 76, 'they': 77, 'life': 78, 'end': 79, 'right': 80, 'some': 81, 'big': 82, 'dead': 83, 'power': 84, 'say': 85, 'white': 86

In [15]:
sequences = list()

# 정수 인코딩

In [22]:
for line in text:
  print(line)
  print(t.texts_to_sequences([line]))
  encoded = t.texts_to_sequences([line])[0] #각 샘플에 대한 정수 인코딩
  print(encoded)
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11]

former nfl cheerleaders settlement offer 1 and a meeting with goodell
[[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116]]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116]
epa to unveil a new rule its effect less science in policymaking
[[100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5, 1119]]
[100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5, 1119]
the new noma explained
[[1, 14, 1120, 1121]]
[1, 14, 1120, 1121]
how a bag of texas dirt  became a times tradition
[[15, 2, 584, 4, 215, 375, 1122, 2, 376, 585]]
[15, 2, 584, 4, 215, 375, 1122, 2, 376, 585]
is school a place for selfexpression
[[8, 167, 2, 136, 6, 1123]]
[8, 167, 2, 136, 6, 1123]
commuter reprogramming
[[1124, 1125]]
[1124, 1125]
ford changed leaders looking for a lift its still looking
[[1126, 1127, 168, 377, 6, 2, 1128, 24, 88, 377]]
[1126, 1127, 168, 377, 6, 2, 1128, 24, 88, 377]
romney failed to win at utah convention but few believe hes doomed
[[1129, 586, 3, 378, 13, 587, 1130, 66, 588, 589, 216, 1131]]
[1129, 586, 3

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [18]:
index_to_word = {}

for key, value in t.word_index.items(): 
  index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [23]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


# Padding

In [24]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
print(sequences[:3])

[[  99  269    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]
 [  99  269  371    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]
 [  99  269  371 1115    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]]


In [47]:
#[n for n in headline if n != "Unknown"]
lambda x: x < 5, range(10)
check_score= lambda x: 'pass' if x>=60 else 'fail'

sequences = np.array(sequences)
x = sequences[:, 1:-1]
y = sequences[:, 0]

In [48]:
print(x[:3])
print(pd.shape(x))

[[ 269    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [ 269  371    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [ 269  371 1115    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]


In [49]:
print(y[:3])

[99 99 99]


In [50]:
y = to_categorical(y, num_classes=vocab_size)

In [40]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
print(vocab_size) #
print(max_len)

# 모델 적용

# LSTM

In [53]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # x = +1 y로 가니 -1 # vocab_size * 10  보통 2의배수
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=50, verbose=2) # 0 = silent, 1 = progress bar,2 = one line per epoch.

Epoch 1/50
976/976 - 29s - loss: 5.7574 - accuracy: 0.0743
Epoch 2/50
976/976 - 28s - loss: 5.2229 - accuracy: 0.0802
Epoch 3/50
976/976 - 27s - loss: 3.8183 - accuracy: 0.2056
Epoch 4/50
976/976 - 27s - loss: 3.2454 - accuracy: 0.2694
Epoch 5/50
976/976 - 27s - loss: 2.9822 - accuracy: 0.2906
Epoch 6/50
976/976 - 27s - loss: 2.7961 - accuracy: 0.3082
Epoch 7/50
976/976 - 27s - loss: 2.6567 - accuracy: 0.3255
Epoch 8/50
976/976 - 27s - loss: 2.5338 - accuracy: 0.3454
Epoch 9/50
976/976 - 28s - loss: 2.4419 - accuracy: 0.3599
Epoch 10/50
976/976 - 27s - loss: 2.2951 - accuracy: 0.3872
Epoch 11/50
976/976 - 27s - loss: 2.1330 - accuracy: 0.4198
Epoch 12/50
976/976 - 27s - loss: 2.0992 - accuracy: 0.4287
Epoch 13/50
976/976 - 27s - loss: 1.8377 - accuracy: 0.4919
Epoch 14/50
976/976 - 28s - loss: 1.6326 - accuracy: 0.5460
Epoch 15/50
976/976 - 28s - loss: 1.4360 - accuracy: 0.6005
Epoch 16/50
976/976 - 27s - loss: 1.3163 - accuracy: 0.6365
Epoch 17/50
976/976 - 27s - loss: 1.1354 - accura

# Simple RNN

In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) # 훈련 조기 종료
mc = ModelCheckpoint('t_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) # 훈련 과정에서 검증 데이터가 가장 높았을 때 저장

In [59]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=50, verbose=2)

Epoch 1/50
976/976 - 58s - loss: 5.5425 - accuracy: 0.0845
Epoch 2/50
976/976 - 14s - loss: 4.6236 - accuracy: 0.1453
Epoch 3/50
976/976 - 14s - loss: 4.7386 - accuracy: 0.1475
Epoch 4/50
976/976 - 14s - loss: 3.8180 - accuracy: 0.2287
Epoch 5/50
976/976 - 14s - loss: 3.4628 - accuracy: 0.2656
Epoch 6/50
976/976 - 14s - loss: 3.2448 - accuracy: 0.2887
Epoch 7/50
976/976 - 14s - loss: 3.1412 - accuracy: 0.2934
Epoch 8/50
976/976 - 14s - loss: 3.3344 - accuracy: 0.2543
Epoch 9/50
976/976 - 14s - loss: 3.2192 - accuracy: 0.2733
Epoch 10/50
976/976 - 14s - loss: 2.7248 - accuracy: 0.3318
Epoch 11/50
976/976 - 14s - loss: 2.6829 - accuracy: 0.3316
Epoch 12/50
976/976 - 14s - loss: 2.7201 - accuracy: 0.3285
Epoch 13/50
976/976 - 14s - loss: 3.8146 - accuracy: 0.2557
Epoch 14/50
976/976 - 14s - loss: 5.7304 - accuracy: 0.0741
Epoch 15/50
976/976 - 14s - loss: 5.6945 - accuracy: 0.0745
Epoch 16/50
976/976 - 14s - loss: 5.6048 - accuracy: 0.0874
Epoch 17/50
976/976 - 14s - loss: 5.0426 - accura